## Задача поиска схожих по смыслу предложений

Условие задачи и данные взяты с курса https://stepik.org/course/102090/

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

Введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec) 
Описание модели из статьи (ссылка есть на странице в гитхаб):  
В качестве датасета для обучения word2vec модели из библиотеки gensim были использованы посты с сайта Stack Overflow, опубликованные с августа 2008 по декабрь 2017 года. Препроцессинг представлял из себя следующий пайплайн: удаление кода и html-тегов; удаление пунктуации (за исключением важных для смысла символов, таких как ‘+’ и ‘#’, которые важны, например, для различения ‘C’, ‘C++’ и ‘C#’); приведение к нижнему регистру; удаление стоп-слов.
Параметры обучения: 3 эпохи, векторы длины 200, окно 5. Итоговая модель представляет собой словарь из 1,787,145 слов.

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-08-20 13:45:41--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: 'SO_vectors_200.bin?download=1'

SO_vectors_200.bin? 100%[===================>]   1.35G  4.27MB/s    in 7m 0s   

2022-08-20 13:52:41 (3.30 MB/s) - 'SO_vectors_200.bin?download=1' saved [1453905423/1453905423]



KeyedVectors - модуль, в котором хранятся готовые векторы из модели. Минус этого модуля - модель невозможно дообучить. Плюс - при использовании готовых векторов требуется меньше оперативной памяти и у них более простой интерфейс работы.

In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)
    print(wv_embeddings[word])

float32 (200,)
[ 0.6851772  -1.2778991  -0.41913974  1.3623164  -3.1675398   0.09950767
  0.6402681  -1.1245339  -0.6699619  -0.6998852   0.4936771  -0.40500194
 -3.0706816  -2.2809966   0.85798043  2.7093108   0.3492745  -0.03494101
 -0.22330493  1.2290467   1.7755157  -3.158358   -0.6515983   0.7224096
  2.3193083  -1.7969862   0.40903398 -2.744604   -1.7179952  -0.914309
 -0.75887376 -0.35140672 -0.5182776  -1.9097351  -0.8300773   0.02147918
  1.1783471   0.03169126 -0.3069023   1.6666299   0.6711357  -2.1706133
 -0.11800487  0.22336982 -1.2075394  -0.86297905 -0.63865614  1.1733794
  0.10022762  0.7017279   2.7290728  -0.4640484  -2.1719306  -0.3562852
 -1.8449957   0.10270727  1.1125596  -0.8364318   1.9513408  -0.97937447
  1.2650859   0.06809282  0.6477318  -0.52431005 -0.6103959  -2.979829
 -0.7889965  -0.11004248  1.7603841  -1.0547444  -0.98998    -1.1834062
 -2.5359967  -0.35286787  0.7733574   0.16300043  0.76991326  1.9223119
 -1.2843752  -1.5023688   0.15765315 -2.415087

In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [13]:
i = 0
for word_info in wv_embeddings.most_similar(positive=['dog'], topn=5):
    boolean = False
    if word_info[0] == 'cat':
        boolean = True
        break
    i+=1
if boolean:
    print('YES', i)
else:
    print('NO')

NO


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [4]:
import numpy as np

import nltk
from nltk.tokenize import word_tokenize

In [131]:
def question_to_vec(question, embeddings=wv_embeddings, dim=200):
    result_vector = np.zeros(dim)
    i = 0
    for word in word_tokenize(question.lower()):
        if word in embeddings:
            result_vector += embeddings[word]
        i+=1
    if i == 0:
        return result_vector
    else:
        return result_vector/i

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [119]:
question_to_vec('I love neural networks')[2].round(2)

-0.96

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 3:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$ (округлите до одного знака после запятой)

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$

In [120]:
1/np.log2(1+9)

0.3010299956639812

$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.  
$dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [125]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    if dup_ranks[0] <= k:
        return 1
    else:
        return 0

In [127]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    if dup_ranks[0] <= k:
        return 1/np.log2(1+dup_ranks[0])
    else:
        return 0

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [11]:
import pandas as pd

In [12]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

In [132]:
vectors_all = []
for sentence in candidates_ranking[0]:
    vectors_all.append(question_to_vec(sentence))
vectors_all

[array([ 0.1765893 , -0.1313546 , -0.31966388, -0.23217212, -0.51753039,
         0.89014448,  0.16898784,  0.08907331, -0.73718435, -0.09982711,
         0.67835199,  0.85759211, -0.57778316,  0.25811653,  0.15175151,
         1.66124574,  0.17429978, -0.59791817,  0.28050682, -0.20293443,
        -0.21785698, -0.69838799, -0.20739181, -0.44873028,  0.666486  ,
         0.59595303, -0.20023556,  0.50139399, -0.22843264,  0.69586237,
         0.61747353,  0.10768632,  0.56025283, -0.38395523, -0.41573644,
         0.23863535,  0.7488566 ,  0.61718451, -0.22468431,  0.87839677,
        -0.70045229,  0.30066439,  0.15707617, -0.15961459, -0.28831927,
        -0.18859252, -0.21841074,  0.02772068,  0.35855705,  0.17943122,
         0.36570346,  0.22957604, -0.43359146, -0.12615395, -0.12007589,
         0.20080114, -0.09917836,  0.66370422,  0.95517467,  0.62222623,
        -0.41973706,  0.03602584,  0.3486517 , -0.38137251, -0.4542412 ,
         0.33639724, -0.45649828,  0.07358115, -0.0

In [129]:
question_to_vec(copy_answers[0])

array([ 4.54814079e-02, -2.32837134e-01,  1.00241024e-01,  1.34567549e+00,
        3.85046609e-01,  1.19057933e+00,  8.90774908e-01, -3.08273897e-01,
       -4.48988187e-01,  2.03043777e-01, -5.60361273e-01, -1.11947393e+00,
        3.91553509e-01,  3.61453216e-01, -6.17516275e-01,  1.06995045e+00,
        1.29471416e-01, -2.91924442e-01, -1.08336313e-01, -4.28425224e-01,
        7.66961896e-02, -2.06590345e-01,  4.05918527e-01,  8.88568278e-02,
        1.65136342e-01,  1.87982455e-01,  7.70720946e-02,  1.44657400e-01,
       -9.04633467e-01,  2.36336422e-01,  2.50996378e-01, -2.16681016e-01,
        1.56717493e-01,  1.66566473e-01, -5.64826924e-01,  5.98381947e-01,
        2.41305026e-01,  5.03646869e-03,  7.88419239e-01, -3.35347286e-01,
       -1.04413124e-03,  4.14756093e-01,  1.87366114e-01,  4.71207658e-01,
        1.47442413e-01, -1.14440419e-01, -4.53942663e-01,  6.91020610e-01,
       -2.32989429e-01, -9.59875400e-01,  1.24588389e+00, -3.92792732e-01,
        4.87822203e-02,  

In [133]:
wv_embeddings.cosine_similarities(question_to_vec(copy_answers[0]), vectors_all)

array([-6.01649682e-04,  1.00000000e+00,  3.76463554e-01, -8.28166768e-02])

In [17]:
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dups_dict = {}
for i in range(len(candidates_ranking[0])):
    dups_dict[i] = wv_embeddings.cosine_similarities(question_to_vec_upd(copy_answers[0]), vectors_all)[i]
dups_dict

{0: -0.000601649682167878,
 1: 0.9999999999999999,
 2: 0.3764635542198165,
 3: -0.08281667676240602}

In [143]:
dup_ranks = []
for i in dups_dict.items():
    if i[1].round() == 1:
        dup_ranks.append(i[0]+1)
dup_ranks

[2]

In [144]:
# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [31]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Переходим к задаче ранжирования

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [21]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line)
    return data

Нам понадобится только файл validation.

In [22]:
validation_data = read_corpus('./data_so/validation.tsv')

Кол-во строк

In [23]:
len(validation_data)

3760

Размер нескольких первых строк

In [24]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54517
2 52818
3 54464
4 53724
5 52297


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# from copy import deepcopy

In [145]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    vectors_all = []
    for sentence in candidates:
        vectors_all.append(question_to_vec_upd(sentence))
    dups_dict_so = {}
    for i in range(len(candidates)):
        dups_dict_so[i] = embeddings.cosine_similarities(question_to_vec_upd(question), vectors_all)[i]
    ranged_dict = dict(sorted(dups_dict_so.items(), key=lambda item: item[1],reverse=True))
    list_of_numbers = []
    list_of_ranged_candidates = []
    for item in ranged_dict:
        list_of_numbers.append(item)
        list_of_ranged_candidates.append(candidates[item])
    return list(zip(list_of_numbers,list_of_ranged_candidates))

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [146]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [147]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (1, 'WPF- How to update the changes in list item of a list'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

In [155]:
for i in rank_candidates(questions[1], candidates[1], wv_embeddings, tokenizer):
    print (i[0], end='')

012

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

Мы возьмем 10 примеров. Это будет быстро, но тем не менее наглядно.

In [45]:
from tqdm.notebook import tqdm

In [84]:
wv_ranking = []
for i, line in enumerate(tqdm(validation_data[:10])):
    q, *ex = line.split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/10 [00:00<?, ?it/s]

Посмотрим, что из себя представляют данные:

In [161]:
line.split('\t')[0]

'How to Send Periodic Ephemeral ("Hidden") Messages From a Slack Bot'

In [80]:
q

'How to print a binary heap tree without recursion?'

Посмотрим на результат ранжирования вопросов:

In [160]:
rank_candidates(q, ex, wv_embeddings, tokenizer)[:5]

[(0, 'Send a message in a channel "only visible to you"'), (733, 'Get emails which were sent with php mail'), (526, 'How can I send an e-mail from a vbs script'), (989, 'How to upload a file received from a POST request to another service using NodeJs'), (772, 'Swiftmailer not send mail by smtp')]

In [85]:
wv_ranking

[4, 27, 3, 4, 8, 2, 3, 71, 1, 1]

In [156]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.431 | Hits@   5: 1.000
DCG@  10: 0.431 | Hits@  10: 1.000
DCG@ 100: 0.431 | Hits@ 100: 1.000
DCG@ 500: 0.431 | Hits@ 500: 1.000
DCG@1000: 0.431 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [88]:
train_data = read_corpus('./data_so/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [111]:
words = [sent.split() for sent in train_data[:100000]]

In [112]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, vector_size=200, min_count=5, window=3).wv

In [113]:
wv_ranking = []
for i, line in enumerate(tqdm(validation_data[:10])):
    q, *ex = line.split('\t')
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
wv_ranking

  0%|          | 0/10 [00:00<?, ?it/s]

[4, 27, 3, 4, 8, 2, 3, 71, 1, 1]

In [157]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.431 | Hits@   5: 1.000
DCG@  10: 0.431 | Hits@  10: 1.000
DCG@ 100: 0.431 | Hits@ 100: 1.000
DCG@ 500: 0.431 | Hits@ 500: 1.000
DCG@1000: 0.431 | Hits@1000: 1.000


## Вывод:  
К сожалению, получение эмебдингов на корпусе похржих вопросов не улучшило качество модели.